In [71]:
import librosa
import os
import numpy as np
import librosa

In [72]:
def make_vector_wav(filename):
    # Load the example clip
    y, sr = librosa.load(filename)

    # Set the hop length; at 22050 Hz, 512 samples ~= 23ms
    hop_length = 512

    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Beat track on the percussive signal
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                                 sr=sr)

    # Compute MFCC features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

    # And the first-order differences (delta features)
    mfcc_delta = librosa.feature.delta(mfcc)

    # Stack and synchronize between beat events
    # This time, we'll use the mean value (default) instead of median
    beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                        beat_frames)

    # Compute chroma features from the harmonic signal
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sr)

    # Aggregate chroma features between beat events
    # We'll use the median value of each feature between beat frames
    beat_chroma = librosa.util.sync(chromagram,
                                    beat_frames,
                                    aggregate=np.median)

    # Finally, stack all beat-synchronous features together
    return np.vstack([beat_chroma, beat_mfcc_delta])

In [73]:
def make_three_demension(X_data):
    max_beats = max(X.shape[1] for X in X_data)
    feature_dim = X_data[0].shape[0]
    num_songs = len(X_data)
    X_data_array = np.zeros((num_songs, feature_dim, max_beats))

    for i, X in enumerate(X_data):
        X_data_array[i, :, :X.shape[1]] = X
    return X_data_array

In [6]:
filepath=r'd:/ai/Downloads/Data/music/Cozy Winter Korean Ballad/'
filelist=os.listdir(filepath)
false_data = []
for filename in filelist:
    false_data.append(make_vector_wav(filepath+'/'+filename))
false_data_array=make_three_demension(false_data)
np.save(r'd:/ai/Downloads/Data/music/false.npy',false_data_array)

NameError: name 'X_data_array' is not defined

In [45]:
filepath=r'd:/ai/Downloads/Data/music/음악'
filelist=os.listdir(filepath)
X_data = []
for filename in filelist:
    X_data.append(make_vector_wav(filepath+'/'+filename))
X_data_array=make_three_demension(X_data)
np.save(r'd:/ai/Downloads/Data/music/음악.npy',X_data_array)

In [74]:
filepath=r'd:/ai/Downloads/Data/music/Melon Daily Top 100 _ 멜론 일간차트/'
filelist=os.listdir(filepath)
false_data = []
for filename in filelist:
    false_data.append(make_vector_wav(filepath+'/'+filename))
false_data_array=make_three_demension(false_data)
np.save(r'd:/ai/Downloads/Data/music/melon.npy',false_data_array)

In [8]:
false_data = np.load(r'd:/ai/Downloads/Data/music/false.npy')
true_data=np.load(r'd:/ai/Downloads/Data/music/음악.npy')

In [77]:
false_data_array.shape, true_data.shape

((88, 38, 888), (171, 38, 1138))

In [79]:
temp=np.zeros((false_data_array.shape[0],false_data_array.shape[1],true_data.shape[2]))
temp[:,:,:false_data_array.shape[2]] = false_data_array
false_data_array = temp
np.save(r'd:/ai/Downloads/Data/music/melon.npy',false_data_array)

In [16]:
X_data=np.concatenate((false_data,true_data), axis=0)

In [17]:
np.save(r'd:/ai/Downloads/Data/music/종속변수.npy',X_data)

In [18]:
X_data.shape

(291, 38, 1138)